In [59]:
import abc
from abc import abstractmethod
from collections import Sequence

In [60]:
class EmptyException(Exception):
    pass

class Stack(metaclass=abc.ABCMeta):

    def __init__(self, data=None):
        data = [] if data is None else data
        assert isinstance(data, (Sequence,))
        self._data = data

    def push(self, e):
        raise NotImplementedError

    def pop(self):
        raise NotImplementedError
    
    def top(self,):
        raise NotImplementedError
    
    def is_empty(self):
        return len(self._data) == 0
    
    def __len__(self):
        return len(self._data)

In [61]:

class ArrayStack(Stack):

    def push(self, e):
        self._data.append(e)

    def top(self):
        if self.is_empty():
            raise EmptyException("The stack is empty")
        return self._data[-1]

    def pop(self):
        if self.is_empty():
            raise EmptyException("The stack is empty")
        return self._data.pop()

    def __repr__(self):
        return str(list(reversed(self._data)))

In [62]:
def is_matched(expr):
    """Algorithm for Matching Delimiters"""
    left, right = "({[", ")}]"
    S = ArrayStack()
    for c in expr:
        if c in left:
            S.push(c)
        elif c in right:
            if S.is_empty():
                return False
            if right.index(c) != left.index(S.pop()):
                return False

    return S.is_empty()


In [63]:
print(is_matched("[(5+x)-(y+z)]"))

True


In [64]:
def is_matched_html(raw: str):
    S = ArrayStack()
    j = raw.find("<")
    while j != -1:
        k = raw.find(">", j+1)
        if k == -1:
            return False
        tag = raw[j+1:k]
        if not tag.startswith("/"):
            S.push(tag)
            print(S)
        else:
            if S.is_empty():
                return False 
            if tag[1:] != S.pop():
                return False
        
        j = raw.find("<", k+1)
        
    return S.is_empty()

is_matched_html("""
<body>
<center>
<h1> The Little Boat </h1>
</center>
<p> The storm tossed the little
boat like a cheap sneaker in an
old washing machine. The three
drunken fishermen were used to
such treatment, of course, but
not the tree salesman, who even as
a stowaway now felt that he
had overpaid for the voyage. </p>
<ol>
<li> Will the salesman die? </li>
<li> What color is the boat? </li>
<li> And what about Naomi? </li>
</ol>
</body>


""")

['body']
['center', 'body']
['h1', 'center', 'body']
['p', 'body']
['ol', 'body']
['li', 'ol', 'body']
['li', 'ol', 'body']
['li', 'ol', 'body']


True

In [65]:
class Queue(abc.ABC):

    def __init__(self, data=None):
        data = [] if data is None else data
        assert isinstance(data, (Sequence,))
        self._data = data

    def enqueue(self, e):
        raise NotImplementedError

    def dequeue(self):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def first(self):
        raise NotImplementedError

    def is_empty(self):
        raise NotImplementedError

    def __repr__(self):
        return str(self._data)


In [66]:
class ArrayQueue(Queue):
    DEFAULT_CAPACITY = 10

    def __init__(self):
        super().__init__( [None] * self.DEFAULT_CAPACITY)
        self.size = 0
        self.front = 0

    def __len__(self):
        return self.size

    def is_empty(self):
        return self.size == 0

    def first(self):
        if self.is_empty():
            raise EmptyException("Queue is empty")
        return self._data[self.front]

    def dequeue(self):
        if self.is_empty():
            raise EmptyException("Queue is empty")
        val = self._data[self.front]
        self._data[self.front] = None
        self.front += 1
        self.front %= len(self._data)
        self.size -= 1
        if 0 < self.size < len(self._data) // 4:
            self.resize(len(self._data) // 2)

        return val

    def enqueue(self, e):
        if self.size == len(self._data):
            self.resize(2 * self.size)
        index = (self.front + self.size) % len(self._data)
        self._data[index] = e
        self.size += 1
        
        
    def resize(self, capacity):
        old = self._data
        self._data = [None] * capacity
        walk = self.front
        for k in range(self.size):
            self._data[k] = old[walk]
            walk = (walk + 1) % len(old)
        self.front = 0

In [67]:
q = ArrayQueue()
q.enqueue(1)
q.enqueue(12)
print(q.dequeue())
print(q.dequeue())
q.enqueue(12)
q.enqueue(14)
q.enqueue(2)
print(q.dequeue())
print(q.dequeue())
print(q.dequeue())
q.enqueue(14)

print(q.dequeue())

1
12
12
14
2
14


In [68]:
class Deque(Queue):
    DEFAULT_CAPACITY = 10
    def __init__(self):
        super().__init__( [None] * self.DEFAULT_CAPACITY)
        self.size = 0
        self.front = 0
        self.rear = 0

    def __len__(self):
        return self.size

    def is_empty(self):
        return self.size == 0

    def add_front(self, e):
        if self.is_empty():
            self._data[self.front] = e
        else:
            if self.size == len(self._data):
                self.resize(2 * self.size)
            self.front = ((self.front + self.size - 1) % len(self._data) +
                         (len(self._data) - self.size)) % len(self._data)
            self._data[self.front] = e
        self.size +=1

    def add_rear(self, e):
        if self.is_empty():
            self._data[self.rear] = e
        else:
            if self.size == len(self._data):
                self.resize(2 * self.size)
            self.rear = (self.front + self.size) % len(self._data)
            self._data[self.rear] = e
        self.size +=1

    def remove_front(self):
        if self.is_empty():
            raise EmptyException("Queue is empty")
        val = self._data[self.front]
        self._data[self.front] = None
        self.front += 1
        self.front %= len(self._data)

        self.size -= 1
        if 0 < self.size < len(self._data) // 4:
            self.resize(len(self._data) // 2)
        return val

    def remove_rear(self):
        if self.is_empty():
            raise EmptyException("Queue is empty")
        val = self._data[self.rear]
        self._data[self.rear] = None
        self.rear -= 1
        self.rear %= len(self._data)
        self.size -= 1
        if 0 < self.size < len(self._data) // 4:
            self.resize(len(self._data) // 2)
        return val

    def last(self):
        if self.is_empty():
            raise EmptyException("Queue is empty")
        return self._data[self.rear]

    def first(self):
        if self.is_empty():
            raise EmptyException("Queue is empty")
        return self._data[self.front]

    def resize(self, capacity):
        old = self._data
        self._data = [None] * capacity
        walk = self.front
        for k in range(self.size):
            self._data[k] = old[walk]
            walk = (walk + 1) % len(old)
        self.front = 0
        self.rear = self.front + self.size - 1

In [69]:
d = Deque()
d.add_front(5)
d.add_front(2)
d.add_rear(3)
d.add_front(4)


In [70]:
# d.add_front(6)
d.remove_rear()

3

In [71]:
def power_set(arr):
    """Use stack S and a queue Q to generate all possible subsets"""
    S = ArrayStack()
    Q = ArrayQueue()
    Q.enqueue(set())
    for el in arr:
        S.push(el)

    while not S.is_empty():
        s = S.pop()
        size = Q.size
        while  size >0:
            qs = Q.dequeue()
            Q.enqueue(qs)
            Q.enqueue(qs | set(s))
            size -= 1
    return [s for s in Q._data if s is not None]


power_set(["A", "B", "C"])


[set(),
 {'A'},
 {'B'},
 {'A', 'B'},
 {'C'},
 {'A', 'C'},
 {'B', 'C'},
 {'A', 'B', 'C'}]

In [72]:
class ArrayQStack(Stack):

    def __init__(self):
        self.q = ArrayQueue()

    def __len__(self):
        return len(self.q)
    @property
    def size(self):
        return self.q.size

    def push(self, e):
        self.q.enqueue(e)

    def pop(self):
        if self.q.is_empty():
            raise EmptyException("The stack is empty")
        if self.q.size == len(self.q._data):
            self.q.resize( 2 * self.q.size)
        last_val = self.q._data[self.q.front + self.size - 1]
        self.q._data[self.q.front + self.size - 1] = None
        self.q.size -= 1
        return last_val

    def top(self,):
        return self.q._data[self.q.front + self.q.size - 1]


In [73]:
q = ArrayQStack()

In [74]:
q.push(2)
q.push(4)
print(q.pop())
q.push(6)
q.push(8)
print(q.pop())
print(q.pop())
print(q.pop())

4
8
6
2


In [80]:
class ParseError(Exception): ...

def postfix_notation(expr):
    s = len(expr)
    i = 0
    opening = "([{"
    closing = ")]}"
    op = "+-*/%"
    S = ArrayStack()
    OpStack = ArrayQStack()
    NStack =  ArrayStack()
    res = []
    while s>0:
        x = expr[i]
        if x in opening:
            S.push(x)
        elif x in closing:
            if closing.index(x) != opening.index(S.pop()):
                raise ParseError
            o = OpStack.pop()
            res.append(o)
        elif x in op:
            OpStack.push(x)
        elif str(x).isdigit():
            res.append(x)
        else:
            raise NotImplementedError
        s -= 1
        i +=1
    
    return " ".join(res)


postfix_notation("((5+2)*(8-3))/4")



'5 2 + 8 3 - * 4'